In [ ]:
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import jieba as jb
import re
import seaborn as sns
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from prettytable import PrettyTable

# Data Loading

In [ ]:
pop = pd.read_csv('data_popular_3.csv')
pop1 = pop.drop(["id", "forumAlias", "commentCount", "likeCount", "createdAt", "excerpt"], axis=1)
label = [1] * pop1.shape[0]
pop1['label'] = label
pop1

,title,content,anonymousSchool,anonymousDepartment,gender,forumName,topics,label
0,好瞎==這算虐待動物吧,更4 B734 \n-\n更3\n雖然整件事讓人氣憤 但希望大家還是保持理性 就事論事 不要...,True,True,F,心情,['虐待動物'],1
1,男友說要禁欲了?哭啊！？,我的的男友，跟我一起破處後，還來不及享受愛愛帶來的快感，就被突然襲來的焦慮嚇到跟我說他要禁欲...,True,True,F,感情,"['男友', '哭']",1
2,H漫網頁被班上喜歡的男同學看到…..,在高三停課的前一週\n班上（坐在我前的男同學）突然有事要聯絡爸爸，但他匆忙出門沒帶手機\n\...,True,True,F,心情,"['心事', '女生', '靠北', '閒聊', '男生']",1
3,意外發現高中朋友當初過世是假的,現在的心情真的很複雜，就像當初聽到她過世的消息一樣突然，事情發生在兩年前，\n前一天晚上她打...,True,True,F,心情,"['朋友', '心事']",1
4,本週連續3位網紅與政府槓上,因為連續看到3位網紅與政府槓上\r\n所以發這篇文\r\n以下按照youtube上傳影片時間...,False,False,M,YouTuber,"['博恩', 'cheap', 'bump', '機車', '政府']",1
...,...,...,...,...,...,...,...,...
559,原來是鹹的,https://megapx-assets.dcard.tw/images/29fef7b3...,False,False,M,VTuber,"['VTuber', 'hololive', 'hololiveEN']",1
560,#分享 theqoo熱門 擅長綜藝的男女偶像TOP2,* 以原PO主觀標準來看的 *\n\n男偶像代表——EXO伯賢\n\n↓\nhttps://...,False,True,F,追星,"['追星', 'SHINee', 'SEVENTEEN', 'ohmygirl', '美珠']",1
561,-近期用餐心得20間濃縮版本(2022/06),一個月內的餐廳(喜歡的小吃)心得濃縮版。\n※業配風波請不要延燒到我這裡，文章內寫不好吃還問...,False,False,F,美食,"['食記', '台北美食']",1
562,10家必吃！網購甜點 不藏私大公開?,安安！我是77??\n第一次在美食版發文有點緊張><\n本身非常愛吃美食還有甜點\n今天來開...,False,False,F,美食,"['網購', '甜點', '食記', '美食', '吃貨']",1


In [ ]:
notp = pd.read_csv('data_notpopular_3.csv')
notp1 = notp.drop(["id", "forumAlias", "commentCount", "likeCount", "createdAt", "excerpt"], axis=1)
label = [0] * notp1.shape[0]
notp1['label'] = label
notp1

,title,content,anonymousSchool,anonymousDepartment,gender,forumName,topics,label
0,無法接受朋友嫁給工人,我和閨蜜a家境都算不錯\n我們倆從小就讀知名國際學校\n朋友圈也都差不多\n大家也常常帶另一...,True,True,F,心情,[],0
1,看熱鬧的小孩撿角,如題，最近還多廟會活動，無意間聽到身邊的長輩說，看熱鬧的小孩這輩子真的撿角，我心裡想說，小朋...,False,True,F,閒聊,"['廟會', '閒聊', '台灣', '時事', '生活']",0
2,渣男,此篇文章為轉貼文章，請更新至最新版本觀看完整內容。,True,True,F,感情,"['渣男', '新竹', '感情', '交友軟體', '女孩']",0
3,男生都愛臭婊子,臭婊男生平常罵歸罵\n其實還是最喜歡這種的吧\n不然為什麼男友一邊說\n我是很乖很棒的女生\...,True,True,F,感情,[],0
4,#開箱 NARUKO茶樹抗痘粉刺寶-消腫痘救星?,NaN,False,False,F,美妝,"['開箱', '茶樹', '保養', '抗痘']",0
...,...,...,...,...,...,...,...,...
562,高醫醫學or北醫牙醫,小弟是南部人，今年有幸正取高醫醫學與北醫牙醫，懇請各路大神給點建議（這兩個選擇的利與弊），感...,False,False,M,考試,"['考試', '高醫', '北醫', '醫學', '學測']",0
563,富邦防疫險*隔離*理賠,因為同住家人確診 所以我們被匡列隔離3+4\n我們是保富邦的 請問有人已理賠隔離費用的嗎！需...,False,False,F,閒聊,"['富邦', '防疫險', '隔離', '閒聊', 'COVID19']",0
564,營業中 顏佑庭,只有我覺得他是被煞到嗎？\n好像六神無主一樣\n送個小碗就那麼兩桌，錢姐桌、棒球桌\n這樣都...,False,True,M,戲劇綜藝,"['營業中', '顏佑庭']",0
565,有人知道他是誰嗎？,很常出現在抖音裡的一群男模走秀！？\n只找到一些關鍵字?中模經紀?\n附上照片 從影片中擷取...,False,False,F,中國明星,"['中國明星', '抖音', '小紅書']",0


In [ ]:
df = pd.concat([pop1,notp1], ignore_index=True)
df

,title,content,anonymousSchool,anonymousDepartment,gender,forumName,topics,label
0,好瞎==這算虐待動物吧,更4 B734 \n-\n更3\n雖然整件事讓人氣憤 但希望大家還是保持理性 就事論事 不要...,True,True,F,心情,['虐待動物'],1
1,男友說要禁欲了?哭啊！？,我的的男友，跟我一起破處後，還來不及享受愛愛帶來的快感，就被突然襲來的焦慮嚇到跟我說他要禁欲...,True,True,F,感情,"['男友', '哭']",1
2,H漫網頁被班上喜歡的男同學看到…..,在高三停課的前一週\n班上（坐在我前的男同學）突然有事要聯絡爸爸，但他匆忙出門沒帶手機\n\...,True,True,F,心情,"['心事', '女生', '靠北', '閒聊', '男生']",1
3,意外發現高中朋友當初過世是假的,現在的心情真的很複雜，就像當初聽到她過世的消息一樣突然，事情發生在兩年前，\n前一天晚上她打...,True,True,F,心情,"['朋友', '心事']",1
4,本週連續3位網紅與政府槓上,因為連續看到3位網紅與政府槓上\r\n所以發這篇文\r\n以下按照youtube上傳影片時間...,False,False,M,YouTuber,"['博恩', 'cheap', 'bump', '機車', '政府']",1
...,...,...,...,...,...,...,...,...
1126,高醫醫學or北醫牙醫,小弟是南部人，今年有幸正取高醫醫學與北醫牙醫，懇請各路大神給點建議（這兩個選擇的利與弊），感...,False,False,M,考試,"['考試', '高醫', '北醫', '醫學', '學測']",0
1127,富邦防疫險*隔離*理賠,因為同住家人確診 所以我們被匡列隔離3+4\n我們是保富邦的 請問有人已理賠隔離費用的嗎！需...,False,False,F,閒聊,"['富邦', '防疫險', '隔離', '閒聊', 'COVID19']",0
1128,營業中 顏佑庭,只有我覺得他是被煞到嗎？\n好像六神無主一樣\n送個小碗就那麼兩桌，錢姐桌、棒球桌\n這樣都...,False,True,M,戲劇綜藝,"['營業中', '顏佑庭']",0
1129,有人知道他是誰嗎？,很常出現在抖音裡的一群男模走秀！？\n只找到一些關鍵字?中模經紀?\n附上照片 從影片中擷取...,False,False,F,中國明星,"['中國明星', '抖音', '小紅書']",0


# Data Preprocessing

In [ ]:
#drop nan
print(df.isnull().sum())
df1=df.dropna()
df1

title                   0
content                54
anonymousSchool         0
anonymousDepartment     0
gender                  0
forumName               0
topics                  0
label                   0
dtype: int64


,title,content,anonymousSchool,anonymousDepartment,gender,forumName,topics,label
0,好瞎==這算虐待動物吧,更4 B734 \n-\n更3\n雖然整件事讓人氣憤 但希望大家還是保持理性 就事論事 不要...,True,True,F,心情,['虐待動物'],1
1,男友說要禁欲了?哭啊！？,我的的男友，跟我一起破處後，還來不及享受愛愛帶來的快感，就被突然襲來的焦慮嚇到跟我說他要禁欲...,True,True,F,感情,"['男友', '哭']",1
2,H漫網頁被班上喜歡的男同學看到…..,在高三停課的前一週\n班上（坐在我前的男同學）突然有事要聯絡爸爸，但他匆忙出門沒帶手機\n\...,True,True,F,心情,"['心事', '女生', '靠北', '閒聊', '男生']",1
3,意外發現高中朋友當初過世是假的,現在的心情真的很複雜，就像當初聽到她過世的消息一樣突然，事情發生在兩年前，\n前一天晚上她打...,True,True,F,心情,"['朋友', '心事']",1
4,本週連續3位網紅與政府槓上,因為連續看到3位網紅與政府槓上\r\n所以發這篇文\r\n以下按照youtube上傳影片時間...,False,False,M,YouTuber,"['博恩', 'cheap', 'bump', '機車', '政府']",1
...,...,...,...,...,...,...,...,...
1126,高醫醫學or北醫牙醫,小弟是南部人，今年有幸正取高醫醫學與北醫牙醫，懇請各路大神給點建議（這兩個選擇的利與弊），感...,False,False,M,考試,"['考試', '高醫', '北醫', '醫學', '學測']",0
1127,富邦防疫險*隔離*理賠,因為同住家人確診 所以我們被匡列隔離3+4\n我們是保富邦的 請問有人已理賠隔離費用的嗎！需...,False,False,F,閒聊,"['富邦', '防疫險', '隔離', '閒聊', 'COVID19']",0
1128,營業中 顏佑庭,只有我覺得他是被煞到嗎？\n好像六神無主一樣\n送個小碗就那麼兩桌，錢姐桌、棒球桌\n這樣都...,False,True,M,戲劇綜藝,"['營業中', '顏佑庭']",0
1129,有人知道他是誰嗎？,很常出現在抖音裡的一群男模走秀！？\n只找到一些關鍵字?中模經紀?\n附上照片 從影片中擷取...,False,False,F,中國明星,"['中國明星', '抖音', '小紅書']",0


In [ ]:
# one label encoding
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
for col in df1[['anonymousSchool','anonymousDepartment','gender', 'forumName']]:
  df1[col]=le.fit_transform(df1[col])
df1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,title,content,anonymousSchool,anonymousDepartment,gender,forumName,topics,label
0,好瞎==這算虐待動物吧,更4 B734 \n-\n更3\n雖然整件事讓人氣憤 但希望大家還是保持理性 就事論事 不要...,1,1,1,42,['虐待動物'],1
1,男友說要禁欲了?哭啊！？,我的的男友，跟我一起破處後，還來不及享受愛愛帶來的快感，就被突然襲來的焦慮嚇到跟我說他要禁欲...,1,1,1,44,"['男友', '哭']",1
2,H漫網頁被班上喜歡的男同學看到…..,在高三停課的前一週\n班上（坐在我前的男同學）突然有事要聯絡爸爸，但他匆忙出門沒帶手機\n\...,1,1,1,42,"['心事', '女生', '靠北', '閒聊', '男生']",1
3,意外發現高中朋友當初過世是假的,現在的心情真的很複雜，就像當初聽到她過世的消息一樣突然，事情發生在兩年前，\n前一天晚上她打...,1,1,1,42,"['朋友', '心事']",1
4,本週連續3位網紅與政府槓上,因為連續看到3位網紅與政府槓上\r\n所以發這篇文\r\n以下按照youtube上傳影片時間...,0,0,2,17,"['博恩', 'cheap', 'bump', '機車', '政府']",1
...,...,...,...,...,...,...,...,...
1126,高醫醫學or北醫牙醫,小弟是南部人，今年有幸正取高醫醫學與北醫牙醫，懇請各路大神給點建議（這兩個選擇的利與弊），感...,0,0,2,98,"['考試', '高醫', '北醫', '醫學', '學測']",0
1127,富邦防疫險*隔離*理賠,因為同住家人確診 所以我們被匡列隔離3+4\n我們是保富邦的 請問有人已理賠隔離費用的嗎！需...,0,0,1,123,"['富邦', '防疫險', '隔離', '閒聊', 'COVID19']",0
1128,營業中 顏佑庭,只有我覺得他是被煞到嗎？\n好像六神無主一樣\n送個小碗就那麼兩桌，錢姐桌、棒球桌\n這樣都...,0,1,2,46,"['營業中', '顏佑庭']",0
1129,有人知道他是誰嗎？,很常出現在抖音裡的一群男模走秀！？\n只找到一些關鍵字?中模經紀?\n附上照片 從影片中擷取...,0,0,1,18,"['中國明星', '抖音', '小紅書']",0


## 清理文字

In [ ]:
#刪除字母、數字和中文以外的字
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line

#刪除一些無意義的常用詞(stopword)
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  


stopwords = stopwordslist("stopword.txt")

In [ ]:
#刪除字母、數字和中文以外的字
for col in ['title','content', 'topics']:
  df1[col] = df1[col].apply(remove_punctuation)
for col in ['title','content', 'topics']:
  df1[col] = df1[col].apply(remove_punctuation)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


清除網址、'B##'和'\n'

In [ ]:
def remove_url(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(r'[http|https]*://[a-zA-Z0-9.?/&=:-]*')
    line = rule.sub('',line)
    return line
    
def remove_other(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(r'\n|B[0-9-]*')
    line = rule.sub('',line)
    return line

In [ ]:
for col in ['title','content', 'topics']:
  df1[col] = df1[col].apply(remove_url)
  df1[col] = df1[col].apply(remove_other)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


分離字詞，並過濾無意義的常用詞(stopword)

In [ ]:
#分離字詞，並過濾無意義的常用詞(stopword)
for col in ['title','content', 'topics']:
  df1[col] = df1[col].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))
df1


Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.254 seconds.
Prefix dict has been built successfully.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,title,content,anonymousSchool,anonymousDepartment,gender,forumName,topics,label
0,瞎 這算 虐待 動物,4 3 整件事 人氣憤 希望 理性 事論 事 人身 攻擊 喔 2 有台 中 老 四川 店家 ...,1,1,1,42,虐待 動物,1
1,男友 禁欲 哭,男友 破處 還來 不及 享受 愛愛帶 快感 襲來 焦慮 嚇到 禁欲 三年 他會 幫 我記 月...,1,1,1,44,男友 哭,1
2,H 漫網 頁 班上 喜歡 男同學,高三 停課 前 一週 班上 坐在 我前 男同學 有事 爸爸 匆忙 出門 沒帶 手機 轉頭問 ...,1,1,1,42,心事 女生 靠北 閒聊 男生,1
3,意外 發現 高中 朋友 當初 過世是 假,心情 複 雜就 當初 聽 過世 消息 事情 發生 兩 年前 前一天 晚上 電話給 太晚 接到...,1,1,1,42,朋友 心事,1
4,本週 連續 3 位 網紅 政府 槓上,連續 3 位 網紅 政府 槓上 發 這篇文 youtube 上傳 影片 時間 排序 1STR...,0,0,2,17,博恩 cheapbump 機車 政府,1
...,...,...,...,...,...,...,...,...
1126,高醫 醫學 or 北醫牙醫,小弟 南部 有幸 正取 高醫 醫學 北醫牙 醫懇 請 各路 大神 給點 建議 這兩個 選擇 ...,0,0,2,98,考試 高 醫北醫 醫學學測,0
1127,富邦 防疫 險隔 離理 賠,同住 家人 確診 匡列 隔離 34 保富邦 請問 已理 賠隔 離費用 資料 時間 大概 多久 謝謝,0,0,1,123,富邦 防疫 險隔 離閒聊 COVID19,0
1128,營業 中 顏佑庭,我覺 煞 好像 六神 無主一樣 送個 小碗 兩桌 錢 姐桌 棒球 桌 送 錯桌 明明 指令 ...,0,1,2,46,營業 中 顏佑庭,0
1129,,很常 抖 音裡 一群 男模 走秀 只 找到 關鍵 字 中模 經紀 附上 照片 影片 中擷 取...,0,0,1,18,中國 明星 抖音 小紅書,0


In [ ]:
df_origin=df1
df1['content'] = df1['title']+ ' '+ df1['content'] + ' ' + df1['topics']
df1 = df1.drop(["title", "topics"], axis=1)
df1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,content,anonymousSchool,anonymousDepartment,gender,forumName,label
0,瞎 這算 虐待 動物 4 3 整件事 人氣憤 希望 理性 事論 事 人身 攻擊 喔 2 有台...,1,1,1,42,1
1,男友 禁欲 哭 男友 破處 還來 不及 享受 愛愛帶 快感 襲來 焦慮 嚇到 禁欲 三年 他...,1,1,1,44,1
2,H 漫網 頁 班上 喜歡 男同學 高三 停課 前 一週 班上 坐在 我前 男同學 有事 爸爸...,1,1,1,42,1
3,意外 發現 高中 朋友 當初 過世是 假 心情 複 雜就 當初 聽 過世 消息 事情 發生 ...,1,1,1,42,1
4,本週 連續 3 位 網紅 政府 槓上 連續 3 位 網紅 政府 槓上 發 這篇文 youtu...,0,0,2,17,1
...,...,...,...,...,...,...
1126,高醫 醫學 or 北醫牙醫 小弟 南部 有幸 正取 高醫 醫學 北醫牙 醫懇 請 各路 大神...,0,0,2,98,0
1127,富邦 防疫 險隔 離理 賠 同住 家人 確診 匡列 隔離 34 保富邦 請問 已理 賠隔 離...,0,0,1,123,0
1128,營業 中 顏佑庭 我覺 煞 好像 六神 無主一樣 送個 小碗 兩桌 錢 姐桌 棒球 桌 送 ...,0,1,2,46,0
1129,很常 抖 音裡 一群 男模 走秀 只 找到 關鍵 字 中模 經紀 附上 照片 影片 中擷 ...,0,0,1,18,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#shuffle data
from sklearn.utils import shuffle

df1=shuffle(df1)
df_origin=shuffle(df_origin)

In [ ]:
# train:test = (all-100):100
train = df1[100:]
test = df1[0:100]
all_data = df1
train

,content,anonymousSchool,anonymousDepartment,gender,forumName,label
1000,我想問 確診 中 確診康 復 沙啞 多久 快 沒聲音 天 請問 隔離 完會 消毒 房間 確診...,0,1,1,6,0
359,分享 減肥必 追 IG 帳號 減肥前 ig 探索 頁面 美食 咖啡店 甜點 每次 美食 克制...,0,1,1,74,1
883,一間旭集 饗 饗 員工 阿肥 吃 過旭集 饗 饗 算是 台灣 規模 巴肥 著員 工手 停下 ...,1,1,2,40,0
984,確診 踢皮球 份 分享 也許 了解 台 中 台灣 防疫 姊姊 一家 三口 最小 女兒 確診 ...,0,0,1,29,0
157,女生 小心 體育 課作業 影片 放到 fb 萬人 社團 意淫 當過 霸 社內 鬼 這次 超過...,1,1,1,42,1
...,...,...,...,...,...,...
396,新聞 Angelababy 賴冠霖 翻拍 韓劇 請 吃 飯 漂亮 姐姐 Angelababy...,0,1,1,114,1
374,我爺爺 送 一台 瑪 莎拉 蒂 事情 天在網 路上 711 賣 賽車 模型 如圖 https...,0,1,2,123,1
925,2022 日本 入境 610 開放 COVID19 日本 旅遊 解禁 簽證 觀光 總 整理 ...,0,0,1,56,0
840,喜歡 趴 趴 兔 兔 第一次 家兔 兔好 可愛喂 吃大便 時 愛 httpsiimgurco...,0,1,2,37,0


In [ ]:
# TF-IDF
count_vect = CountVectorizer()
train_x = count_vect.fit_transform(train['content'])
test_x = count_vect.transform(test['content'])

tfidf_transformer = TfidfTransformer()
train_x = tfidf_transformer.fit_transform(train_x)
test_x = tfidf_transformer.transform(test_x)



In [ ]:
train_y=train['label'].to_numpy()
test_y=test['label'].to_numpy()
test_y.shape

#all
all_y=df1['label'].to_numpy()

# Model

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from prettytable import PrettyTable
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.model_selection import train_test_split

### MLP

In [ ]:
from scipy.sparse import coo_matrix,hstack
from sklearn import preprocessing

In [ ]:
lb_anonymousSchool = preprocessing.LabelBinarizer(sparse_output=True)
anonymousSchool = lb_anonymousSchool.fit_transform(all_data['anonymousSchool'])

lb_anonymousDepartment = preprocessing.LabelBinarizer(sparse_output=True)
anonymousDepartment = lb_anonymousDepartment.fit_transform(all_data['anonymousDepartment'])

lb_gender = preprocessing.LabelBinarizer(sparse_output=True)
gender = lb_gender.fit_transform(all_data['gender'])

lb_forumName = preprocessing.LabelBinarizer(sparse_output=True)
forumName = lb_forumName.fit_transform(all_data['forumName'])


In [ ]:
tfidf_content = CountVectorizer()
content = tfidf_content.fit_transform(all_data['content'])

tfidf_transformer = TfidfTransformer()
content = tfidf_transformer.fit_transform(content)

In [ ]:
all_x = hstack((anonymousSchool, anonymousDepartment, gender, forumName, content)).tocsr() #0.68


#train/test
train_x = all_x[100:]
test_x = all_x[0:100]


In [ ]:
#transform to np array
x=train_x.toarray()
y=np.array(train_y)

## 取不同種類的feature

In [ ]:
all_data=df_origin

In [ ]:
lb_anonymousSchool = preprocessing.LabelBinarizer(sparse_output=True)
anonymousSchool = lb_anonymousSchool.fit_transform(all_data['anonymousSchool'])

lb_anonymousDepartment = preprocessing.LabelBinarizer(sparse_output=True)
anonymousDepartment = lb_anonymousDepartment.fit_transform(all_data['anonymousDepartment'])

lb_gender = preprocessing.LabelBinarizer(sparse_output=True)
gender = lb_gender.fit_transform(all_data['gender'])

lb_forumName = preprocessing.LabelBinarizer(sparse_output=True)
forumName = lb_forumName.fit_transform(all_data['forumName'])


In [ ]:
tfidf_content = CountVectorizer()
content = tfidf_content.fit_transform(all_data['title']+ ' ' + all_data['topics'])

tfidf_transformer = TfidfTransformer()
content = tfidf_transformer.fit_transform(content)

1. anonymousSchool, anonymousDepartment, gender, forumName, title, topics

In [ ]:
all_x = hstack((anonymousSchool, anonymousDepartment, gender, forumName, content)).tocsr() #0.68

train_x_1 = all_x[100:]
test_x_1 = all_x[0:100]

train_y=all_data[100:]['label'].to_numpy()
test_y=all_data[0:100]['label'].to_numpy()

x=train_x_1.toarray()
y=np.array(train_y)

In [ ]:
#MLP2

model2 = Sequential()
model2.add(Dense(128, input_dim=all_x.shape[1], activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(32, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))

model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               453120    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 3

In [ ]:
#compile
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#Training
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

history2_1=model2.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=15)

Epoch 1/15
22/22 [==============================] - 2s 22ms/step - loss: 0.6934 - accuracy: 0.5110 - val_loss: 0.6918 - val_accuracy: 0.5000
Epoch 2/15
22/22 [==============================] - 0s 10ms/step - loss: 0.6854 - accuracy: 0.5769 - val_loss: 0.6869 - val_accuracy: 0.6122
Epoch 3/15
22/22 [==============================] - 0s 10ms/step - loss: 0.6791 - accuracy: 0.5827 - val_loss: 0.6795 - val_accuracy: 0.6122
Epoch 4/15
22/22 [==============================] - 0s 12ms/step - loss: 0.6546 - accuracy: 0.6691 - val_loss: 0.6442 - val_accuracy: 0.6871
Epoch 5/15
22/22 [==============================] - 0s 12ms/step - loss: 0.5914 - accuracy: 0.7247 - val_loss: 0.5661 - val_accuracy: 0.7687
Epoch 6/15
22/22 [==============================] - 0s 13ms/step - loss: 0.4636 - accuracy: 0.8258 - val_loss: 0.4861 - val_accuracy: 0.7755
Epoch 7/15
22/22 [==============================] - 0s 12ms/step - loss: 0.3464 - accuracy: 0.8799 - val_loss: 0.4383 - val_accuracy: 0.7823
Epoch 8/15
22

In [ ]:
mlp_predict=model2.predict(test_x_1)
for id, p in enumerate(mlp_predict):
  if p>=0.5:
    mlp_predict[id]=1
  else:
    mlp_predict[id]=0
acc_mlp1=accuracy_score(test_y,mlp_predict)

print(acc_mlp1)

0.89


2. anonymousSchool, anonymousDepartment, gender, title, topics

In [ ]:
all_x = hstack((anonymousSchool, anonymousDepartment, gender, content)).tocsr() #0.68

train_x_1 = all_x[100:]
test_x_1 = all_x[0:100]

train_y=all_data[100:]['label'].to_numpy()
test_y=all_data[0:100]['label'].to_numpy()

x=train_x_1.toarray()
y=np.array(train_y)

In [ ]:
#MLP2

model2 = Sequential()
model2.add(Dense(128, input_dim=all_x.shape[1], activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(32, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))

model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               436224    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                

In [ ]:
#compile
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#Training
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

history2_1=model2.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=15)

Epoch 1/15
22/22 [==============================] - 1s 20ms/step - loss: 0.6916 - accuracy: 0.5505 - val_loss: 0.6914 - val_accuracy: 0.5952
Epoch 2/15
22/22 [==============================] - 0s 13ms/step - loss: 0.6897 - accuracy: 0.5403 - val_loss: 0.6904 - val_accuracy: 0.6054
Epoch 3/15
22/22 [==============================] - 0s 12ms/step - loss: 0.6906 - accuracy: 0.5286 - val_loss: 0.6868 - val_accuracy: 0.6497
Epoch 4/15
22/22 [==============================] - 0s 10ms/step - loss: 0.6755 - accuracy: 0.6237 - val_loss: 0.6742 - val_accuracy: 0.7211
Epoch 5/15
22/22 [==============================] - 0s 12ms/step - loss: 0.6505 - accuracy: 0.7013 - val_loss: 0.6420 - val_accuracy: 0.7687
Epoch 6/15
22/22 [==============================] - 0s 14ms/step - loss: 0.5765 - accuracy: 0.7628 - val_loss: 0.5785 - val_accuracy: 0.7687
Epoch 7/15
22/22 [==============================] - 0s 12ms/step - loss: 0.4554 - accuracy: 0.8433 - val_loss: 0.5130 - val_accuracy: 0.7551
Epoch 8/15
22

In [ ]:
mlp_predict=model2.predict(test_x_1)
for id, p in enumerate(mlp_predict):
  if p>=0.5:
    mlp_predict[id]=1
  else:
    mlp_predict[id]=0
acc_mlp2=accuracy_score(test_y,mlp_predict)

print(acc_mlp2)

0.79


3. anonymousSchool, anonymousDepartment, title, topics

In [ ]:
all_x = hstack((anonymousSchool, anonymousDepartment, content)).tocsr() #0.68

train_x_1 = all_x[100:]
test_x_1 = all_x[0:100]

train_y=all_data[100:]['label'].to_numpy()
test_y=all_data[0:100]['label'].to_numpy()

x=train_x_1.toarray()
y=np.array(train_y)

In [ ]:
#MLP2

model2 = Sequential()
model2.add(Dense(128, input_dim=all_x.shape[1], activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(32, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))

model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 128)               435840    
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dropout_8 (Dropout)         (None, 32)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                

In [ ]:
#compile
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#Training
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

history2_1=model2.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=15)

Epoch 1/15
22/22 [==============================] - 1s 16ms/step - loss: 0.6926 - accuracy: 0.5212 - val_loss: 0.6921 - val_accuracy: 0.5102
Epoch 2/15
22/22 [==============================] - 0s 11ms/step - loss: 0.6891 - accuracy: 0.5198 - val_loss: 0.6910 - val_accuracy: 0.4898
Epoch 3/15
22/22 [==============================] - 0s 10ms/step - loss: 0.6817 - accuracy: 0.5417 - val_loss: 0.6871 - val_accuracy: 0.4966
Epoch 4/15
22/22 [==============================] - 0s 11ms/step - loss: 0.6611 - accuracy: 0.5710 - val_loss: 0.6747 - val_accuracy: 0.5680
Epoch 5/15
22/22 [==============================] - 0s 10ms/step - loss: 0.6213 - accuracy: 0.7086 - val_loss: 0.6404 - val_accuracy: 0.6871
Epoch 6/15
22/22 [==============================] - 0s 10ms/step - loss: 0.5175 - accuracy: 0.8243 - val_loss: 0.5828 - val_accuracy: 0.7041
Epoch 7/15
22/22 [==============================] - 0s 9ms/step - loss: 0.3462 - accuracy: 0.9268 - val_loss: 0.5468 - val_accuracy: 0.7381
Epoch 8/15
22/

In [ ]:
mlp_predict=model2.predict(test_x_1)
for id, p in enumerate(mlp_predict):
  if p>=0.5:
    mlp_predict[id]=1
  else:
    mlp_predict[id]=0
acc_mlp3=accuracy_score(test_y,mlp_predict)


print(acc_mlp3)

0.81


4. anonymousSchool, anonymousDepartment, forumName, title, topics

In [ ]:
all_x = hstack((anonymousSchool, anonymousDepartment, forumName, content)).tocsr() #0.68

train_x_1 = all_x[100:]
test_x_1 = all_x[0:100]

train_y=all_data[100:]['label'].to_numpy()
test_y=all_data[0:100]['label'].to_numpy()

x=train_x_1.toarray()
y=np.array(train_y)

In [ ]:
#MLP2

model2 = Sequential()
model2.add(Dense(128, input_dim=all_x.shape[1], activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(32, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))

model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 128)               452736    
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_14 (Dense)            (None, 32)                2080      
                                                                 
 dropout_11 (Dropout)        (None, 32)                0         
                                                                 
 dense_15 (Dense)            (None, 1)                

In [ ]:
#compile
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#Training
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

history2_1=model2.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=15)

Epoch 1/15
22/22 [==============================] - 1s 24ms/step - loss: 0.6908 - accuracy: 0.5212 - val_loss: 0.6914 - val_accuracy: 0.4796
Epoch 2/15
22/22 [==============================] - 0s 14ms/step - loss: 0.6850 - accuracy: 0.5505 - val_loss: 0.6872 - val_accuracy: 0.4796
Epoch 3/15
22/22 [==============================] - 0s 12ms/step - loss: 0.6733 - accuracy: 0.5681 - val_loss: 0.6731 - val_accuracy: 0.5272
Epoch 4/15
22/22 [==============================] - 0s 13ms/step - loss: 0.6352 - accuracy: 0.6413 - val_loss: 0.6292 - val_accuracy: 0.7007
Epoch 5/15
22/22 [==============================] - 0s 13ms/step - loss: 0.5650 - accuracy: 0.7643 - val_loss: 0.5508 - val_accuracy: 0.7483
Epoch 6/15
22/22 [==============================] - 0s 12ms/step - loss: 0.4346 - accuracy: 0.8243 - val_loss: 0.4656 - val_accuracy: 0.7959
Epoch 7/15
22/22 [==============================] - 0s 13ms/step - loss: 0.2963 - accuracy: 0.9209 - val_loss: 0.4630 - val_accuracy: 0.7925
Epoch 8/15
22

In [ ]:
mlp_predict=model2.predict(test_x_1)
for id, p in enumerate(mlp_predict):
  if p>=0.5:
    mlp_predict[id]=1
  else:
    mlp_predict[id]=0
acc_mlp4=accuracy_score(test_y,mlp_predict)


print(acc_mlp4)

0.85


5. anonymousSchool, forumName, title, topics

In [ ]:
all_x = hstack((anonymousSchool, forumName, content)).tocsr() #0.68

train_x_1 = all_x[100:]
test_x_1 = all_x[0:100]

train_y=all_data[100:]['label'].to_numpy()
test_y=all_data[0:100]['label'].to_numpy()

x=train_x_1.toarray()
y=np.array(train_y)

In [ ]:
#MLP2

model2 = Sequential()
model2.add(Dense(128, input_dim=all_x.shape[1], activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(32, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))

model2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 128)               452608    
                                                                 
 dropout_12 (Dropout)        (None, 128)               0         
                                                                 
 dense_17 (Dense)            (None, 64)                8256      
                                                                 
 dropout_13 (Dropout)        (None, 64)                0         
                                                                 
 dense_18 (Dense)            (None, 32)                2080      
                                                                 
 dropout_14 (Dropout)        (None, 32)                0         
                                                                 
 dense_19 (Dense)            (None, 1)                

In [ ]:
#compile
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#Training
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

history2_1=model2.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=15)

Epoch 1/15
22/22 [==============================] - 1s 19ms/step - loss: 0.6931 - accuracy: 0.4949 - val_loss: 0.6879 - val_accuracy: 0.6633
Epoch 2/15
22/22 [==============================] - 0s 11ms/step - loss: 0.6850 - accuracy: 0.5915 - val_loss: 0.6785 - val_accuracy: 0.6565
Epoch 3/15
22/22 [==============================] - 0s 13ms/step - loss: 0.6709 - accuracy: 0.6032 - val_loss: 0.6549 - val_accuracy: 0.7211
Epoch 4/15
22/22 [==============================] - 0s 11ms/step - loss: 0.6241 - accuracy: 0.7204 - val_loss: 0.5996 - val_accuracy: 0.7313
Epoch 5/15
22/22 [==============================] - 0s 11ms/step - loss: 0.5355 - accuracy: 0.8009 - val_loss: 0.5112 - val_accuracy: 0.7755
Epoch 6/15
22/22 [==============================] - 0s 10ms/step - loss: 0.4035 - accuracy: 0.8960 - val_loss: 0.4470 - val_accuracy: 0.7925
Epoch 7/15
22/22 [==============================] - 0s 10ms/step - loss: 0.2605 - accuracy: 0.9341 - val_loss: 0.4543 - val_accuracy: 0.8061
Epoch 8/15
22

In [ ]:
mlp_predict=model2.predict(test_x_1)
for id, p in enumerate(mlp_predict):
  if p>=0.5:
    mlp_predict[id]=1
  else:
    mlp_predict[id]=0
acc_mlp5=accuracy_score(test_y,mlp_predict)


print(acc_mlp5)

0.87


6. forumName, title, topics

In [ ]:
all_x = hstack((forumName, content)).tocsr() #0.68

train_x_1 = all_x[100:]
test_x_1 = all_x[0:100]

train_y=all_data[100:]['label'].to_numpy()
test_y=all_data[0:100]['label'].to_numpy()

x=train_x_1.toarray()
y=np.array(train_y)

In [ ]:
#MLP2

model2 = Sequential()
model2.add(Dense(128, input_dim=all_x.shape[1], activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(32, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))

model2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 128)               452480    
                                                                 
 dropout_15 (Dropout)        (None, 128)               0         
                                                                 
 dense_21 (Dense)            (None, 64)                8256      
                                                                 
 dropout_16 (Dropout)        (None, 64)                0         
                                                                 
 dense_22 (Dense)            (None, 32)                2080      
                                                                 
 dropout_17 (Dropout)        (None, 32)                0         
                                                                 
 dense_23 (Dense)            (None, 1)                

In [ ]:
#compile
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#Training
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

history2_1=model2.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=15)

Epoch 1/15
22/22 [==============================] - 1s 18ms/step - loss: 0.6936 - accuracy: 0.4905 - val_loss: 0.6901 - val_accuracy: 0.4660
Epoch 2/15
22/22 [==============================] - 0s 11ms/step - loss: 0.6799 - accuracy: 0.5930 - val_loss: 0.6804 - val_accuracy: 0.5374
Epoch 3/15
22/22 [==============================] - 0s 13ms/step - loss: 0.6593 - accuracy: 0.6047 - val_loss: 0.6601 - val_accuracy: 0.5510
Epoch 4/15
22/22 [==============================] - 0s 16ms/step - loss: 0.6090 - accuracy: 0.7057 - val_loss: 0.6112 - val_accuracy: 0.7007
Epoch 5/15
22/22 [==============================] - 0s 15ms/step - loss: 0.5274 - accuracy: 0.8375 - val_loss: 0.5489 - val_accuracy: 0.7415
Epoch 6/15
22/22 [==============================] - 0s 14ms/step - loss: 0.3803 - accuracy: 0.8931 - val_loss: 0.5002 - val_accuracy: 0.7517
Epoch 7/15
22/22 [==============================] - 0s 13ms/step - loss: 0.2301 - accuracy: 0.9531 - val_loss: 0.5054 - val_accuracy: 0.7687
Epoch 8/15
22

In [ ]:
mlp_predict=model2.predict(test_x_1)
for id, p in enumerate(mlp_predict):
  if p>=0.5:
    mlp_predict[id]=1
  else:
    mlp_predict[id]=0
acc_mlp6=accuracy_score(test_y,mlp_predict)


print(acc_mlp6)

0.88


In [ ]:
#Accuracy
print("Accuracy")
table=PrettyTable(['anonymousSchool', 'anonymousDepartment', 'gender', 'forumName', 'title', 'topics', 'Accuracy'])
# anonymousSchool, anonymousDepartment, gender, forumName, title, topics
table.add_row(['v', 'v', 'v', 'v', 'v', 'v', acc_mlp1])
table.add_row(['v', 'v', 'v', 'x', 'v', 'v', acc_mlp2])
table.add_row(['v', 'v', 'x', 'x', 'v', 'v', acc_mlp3])
table.add_row(['v', 'v', 'x', 'v', 'v', 'v', acc_mlp4])
table.add_row(['v', 'x', 'x', 'v', 'v', 'v', acc_mlp5])
table.add_row(['x', 'x', 'x', 'v', 'v', 'v', acc_mlp6])
print(table)


Accuracy
+-----------------+---------------------+--------+-----------+-------+--------+----------+
| anonymousSchool | anonymousDepartment | gender | forumName | title | topics | Accuracy |
+-----------------+---------------------+--------+-----------+-------+--------+----------+
|        v        |          v          |   v    |     v     |   v   |   v    |   0.89   |
|        v        |          v          |   v    |     x     |   v   |   v    |   0.79   |
|        v        |          v          |   x    |     x     |   v   |   v    |   0.81   |
|        v        |          v          |   x    |     v     |   v   |   v    |   0.85   |
|        v        |          x          |   x    |     v     |   v   |   v    |   0.87   |
|        x        |          x          |   x    |     v     |   v   |   v    |   0.88   |
+-----------------+---------------------+--------+-----------+-------+--------+----------+
